# Rainfall Prediction Project

## Data Loading, combining and EDA

*Group 12*

------------

In [ ]:
import re
import os
import glob
import zipfile
import requests
from urllib.request import urlretrieve
import json
import pandas as pd
import rpy2.rinterface
import altair as alt

## Download the data

In [ ]:
article_id = 14096681
url = f"https://api.figshare.com/v2/articles/{article_id}"
headers = {"Content-Type": "application/json"}
output_directory = "../data/raw/figsharerainfall/"

In [ ]:
response = requests.request("GET", url, headers=headers)
data = json.loads(response.text)
files = data["files"]

In [ ]:
files_to_dl = ["data.zip"]
for file in files:
    if file["name"] in files_to_dl:
        os.makedirs(output_directory, exist_ok=True)
        urlretrieve(file["download_url"], output_directory + file["name"])

In [ ]:
with zipfile.ZipFile(os.path.join(output_directory, files_to_dl[0]), 'r') as f:
    f.extractall(output_directory)

## Combine the data

In [ ]:
%%time
exclude = "observed_daily_rainfall_SYD.csv"
files = glob.glob('../data/raw/figsharerainfall/*.csv')
df = pd.concat(
    (pd.read_csv(file, index_col=0)
                .assign(model=re.findall(r'[A-Z][^_]+', file)[0])
                for file in files if file is not exclude)
)
df.to_csv("../data/processed/combined_data.csv")

In [ ]:
df = pd.read_csv("../data/processed/combined_data.csv")
df.head()

In [ ]:
df.tail()

## Combine data csv on different machines

- Compare observations:

| Team Member   | Operating System | RAM | Processor | Is SSD | Time taken |
|:-------------:|:----------------:|:---:|:---------:|:------:|:----------:|
| Vera Cui      | macOS            | 16GB| M1        | No     | 6min 39s   |
| Lynn Wu       | macOS            | 8GB | M1        | Yes    |  6min 7s   |
| Jasmine Ortega|  macOS         |  8GB |   M1   |  Yes   | 9min 56s     |
| Maeve Shi   | MacOS Big Sur    | 8GB | 2.3 GHz Dual-Core Intel Core i5 | Yes |  7min 30s   |

--------------

##  Load csv and perform EDA on different machines

#### Baseline `read_csv` time

In [ ]:
%%time
df = pd.read_csv("../data/processed/combined_data.csv")

In [ ]:
df.info()

| Team Member   | Operating System | RAM | Processor | Is SSD | Time taken |
|:-------------:|:----------------:|:---:|:---------:|:------:|:----------:|
| Vera Cui      |                  |     |           |        |            |
| Lynn Wu       |                  |     |           |        |            |
| Jasmine Ortega|  MacOS           |8GB  |    M1     |  yes   |1 min 9s    |
| Yike Shi      |                  |     |           |        |            |

As is, the csv file took 1 minute and 46 seconds to load. From `.info()` we can see that the df consists of 6 columns all of the dtype `float64`. To reduce memory usage, we will first convert the data type to `float32` and `float16`, both which will reduce memory used, as shown below.

In [ ]:
print(f"Memory usage with float64: {df.memory_usage().sum() / 1e6:.2f} MB")
print(f"Memory usage with float32: {df.astype('float32', errors='ignore').memory_usage().sum() / 1e6:.2f} MB")
print(f"Memory usage with float16: {df.astype('float16', errors='ignore').memory_usage().sum() / 1e6:.2f} MB")

#### Approaches to reduce memory usage while performing the EDA: *changing datatype*
##### Convert to float32

In [ ]:
%%time
dtypes = {"lat_min" : "float32",
         "lat_max" : "float32",
         "lon_min" : "float32",
         "lon_max" : "float32",
         "rain (mm/day)" : "float32",
          "model" : "string"
        }

df_float32 = pd.read_csv('../data/processed/combined_data.csv', dtype=dtypes)

In [ ]:
df_float32.info()

##### Convert to float16

In [ ]:
%%time
dtypes = {"lat_min" : "float16",
         "lat_max" : "float16",
         "lon_min" : "float16",
         "lon_max" : "float16",
         "rain (mm/day)" : "float16",
        "model" : "string"
        }

df_float16 = pd.read_csv('../data/processed/combined_data.csv', dtype=dtypes)

In [ ]:
df_float16 

As demonstrated, changing `float64` to less precise datatypes reduced runtimes. Interestingly, it looks like `float16` (50s) took almost as long to load as the more precise `float32` (58s). 

#### Approaches to reduce memory usage while performing the EDA: *Loading in chunks*

In [ ]:
%%time

chunk = pd.read_csv("../data/processed/combined_data.csv", chunksize=10_000_000, iterator=True)
df = pd.concat(chunk)
df

Loading the data in chunks of 10,000,000 reduced the loading time to 1min 7s. Let's combine the `float32` strategy with loading in chunks! (Note: we use `float32` instead of `float16` because half precision types are not supported when loading the data in chunks.  

In [ ]:
%%time

dtypes = {"lat_min" : "float32",
         "lat_max" : "float32",
         "lon_min" : "float32",
         "lon_max" : "float32",
         "rain (mm/day)" : "float32",
         "model" : "string"
        }

final_df = pd.DataFrame()

chunk = pd.read_csv("../data/processed/combined_data.csv", chunksize=10_000_000, iterator=True, dtype=dtypes)
final_df = pd.concat(chunk)

In [ ]:
final_df.info()

We successfully reduced the load time from 1min 9s to 54.9s. 


**Optimized data loading:**

| Team Member   | Operating System | RAM | Processor | Is SSD | Time taken |
|:-------------:|:----------------:|:---:|:---------:|:------:|:----------:|
| Vera Cui      |                  |     |           |        |            |
| Lynn Wu       |                  |     |           |        |            |
| Jasmine Ortega|  MacOS           |8GB  |    M1     |  yes   |54.9s       |
| Yike Shi      |                  |     |           |        |            |

Now, we will load the data (optimized) and conduct a simple EDA in which we will count the number of each model in the dataframe. 

In [ ]:
%%time

dtypes = {"lat_min" : "float32",
         "lat_max" : "float32",
         "lon_min" : "float32",
         "lon_max" : "float32",
         "rain (mm/day)" : "float32",
         "model" : "string"
        }

final_df = pd.DataFrame()

chunk = pd.read_csv("../data/processed/combined_data.csv", chunksize=10_000_000, iterator=True, dtype=dtypes)
final_df = pd.concat(chunk)

# EDA
model_count = final_df["model"].value_counts()
model_count

**Python data load + EDA time:**

| Team Member   | Operating System | RAM | Processor | Is SSD | Time taken |
|:-------------:|:----------------:|:---:|:---------:|:------:|:----------:|
| Vera Cui      |                  |     |           |        |            |
| Lynn Wu       |                  |     |           |        |            |
| Jasmine Ortega|  MacOS           |8GB  |    M1     |  yes   |58.5 s      |
| Yike Shi      |                  |     |           |        |            |

It looks like it takes ~1 minute to load and execute the EDA using the datatype `float32` and loading in chunks of 10,000,0000. 

(We attempted to plot the value counts, however, it consistently crashed the kernel on Mac M1) 

In [ ]:
# alt.data_transformers.disable_max_rows()
# alt.renderers.enable('mimetype')

In [ ]:
# this kills my kernel but you can try it lol 
# count_plot = alt.Chart(final_df).mark_bar().encode(x='model', y='count()')# this kills my kernel but you can try it lol 
# count_plot

#### Transform Python df to R Parquet 

We chose to use a parquet file to transfer the dataframe from Python to R because parquet files work well with rarrow. rarrow is ideal because by default, it reads and processes data in chunks, which we saw greatly improved our CSV loading time.

In [ ]:
final_df.to_parquet("../data/processed/combined_data.parquet")

#### Perform a simple EDA in R

In [ ]:
%load_ext rpy2.ipython

In [ ]:
%%time 
%%R
library(dplyr)
library(arrow)


df <- open_dataset("../data/processed/combined_data.parquet") 

eda <- df |> count(model)
eda

result <- eda %>% collect
print(result, n=28)

**R data load + EDA time:**

| Team Member   | Operating System | RAM | Processor | Is SSD | Time taken |
|:-------------:|:----------------:|:---:|:---------:|:------:|:----------:|
| Vera Cui      |                  |     |           |        |            |
| Lynn Wu       |                  |     |           |        |            |
| Jasmine Ortega|  MacOS           |8GB  |    M1     |  yes   |2.49 s      |
| Yike Shi      |                  |     |           |        |            |

Overall, using a parquet file to transfer a Python dataframe to R was extremely effective. Considering the sheer volume of data we have (62 million rows), it's amazing that R was able to compute value counts in 2.5 seconds! Compare to Python, which took 58.5 seconds to complete the same task. From this simple experiment alone, it is clear to see the value of using an optimized storage file like a parquet over a CSV. 

In [ ]:
# installing tidyverse took years so im not sure if this is executable either 
# %%R

# library(ggplot2)
# library(tidyverse)

# count_eda <- result |>
#             groupby(model) |>
#             mutate(count = n())
            
# count_eda